In [12]:
import datetime 
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pymongo
import warnings
from dateutil.parser import parse
import datetime as dt
from lxml import etree
import re


class DataScraper:
    
    def __init__(self):
        warnings.filterwarnings("ignore")
        client = pymongo.MongoClient('127.0.0.1', 27017)
        db = client["news"]
        newsCollection = db["news_collection"]

        self.proxies = {
            "https" : "fnyproxy.fnylocal.com:8080",
            "http" : "fnyproxy.fnylocal.com:8080"
        }

        self.headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36 Edg/126.0.0.0'}
        
        
        
    
    def soup_maker(self, url):

        res = requests.get(url, proxies = self.proxies,headers=self.headers, verify = False)

        soup = BeautifulSoup(res.content.decode('utf-8'), "lxml")
        return soup
    
    def turkce_aylar(self, ay):
        ham_ay = ay[2:len(ay)-5]
        turkce = re.sub('[0-9 ]', '', ay)
        ing_dict = {'Ocak': ' January', 'Şubat': ' February', 'Mart': ' March', 'Nisan': ' April', 'Mayıs': ' May', 'Haziran': ' June', 'Temmuz': ' July', 'Ağustos': ' August',
            'Eylül': ' September', 'Ekim': ' October', 'Kasım': ' November', 'Aralık': ' December' }

        return ay.replace(ham_ay,ing_dict[turkce])
        
    ####### AA ####### //div/div/header/h1[contains(@class, "text-3xl font-medium tracking-tight text-gray-1100 leading-relative-2 md:text-5xl")]
    def aa_news_links(self):
        
        
        url = "https://www.aa.com.tr/tr/ekonomi"



        soup = self.soup_maker(url)

        dom = etree.HTML(str(soup))

        
        links_element = dom.xpath('//div/div[contains(@class, "row konu-alt-icerik")]/a')[:-1]

        new_rows_df = pd.DataFrame(columns = ["headline", "alltext", "date", "source", "url"])

        for link_ele in links_element:
            combine_whole_link =  link_ele.get('href')
            print(combine_whole_link)
            
            title, text, date = self.go_inside_aa_links(link = combine_whole_link)
            
            new_row = {"headline": title, "alltext": text, "date": date, "source": "AA", "url": combine_whole_link}
            new_rows_df = new_rows_df.append(new_row, ignore_index=True)

        return new_rows_df
    
    
    def go_inside_aa_links(self, link):
        # for link in links: 
        soup = self.soup_maker(link)
        dom = etree.HTML(str(soup))

        title_element = dom.xpath('//div/div[contains(@class, "detay-spot-category")]/h1/text()')
        if title_element != []:
            title = title_element[0]
        else:
            title = title_element
        # title = 'titlegelecek'
        
        text_element =dom.xpath('//div/div[contains(@class, "detay-icerik")]/div/p/text()')
        text = ''
        self.text_element = text_element
        for paragraph in text_element:
            text = text + paragraph
        # text= 'textgelecek'

        date_element = dom.xpath('//div/div[contains(@class, "detay-spot-category")]/div/span[contains(@class, "tarih")]/text()')[0]
        date = date_element
        date = date[-10:]
        
        monthDict={"01":'Jan', "02":'Feb', "03":'Mar', "04":'Apr', "05":'May', "06":'Jun', "07":'Jul', "08":'Aug',"09":'Sep', "10":'Oct', "11":'Nov', "12":'Dec'}

        ay_num = date.split('.')[1]

        date = sda.replace(ay_num, monthDict[ay_num])
        

        
        date = pd.to_datetime(date)
        date = date.strftime('%Y-%d-%m-%H-%M-%S')

        print(date)
        # date = '13'


        return title, text, date
            
        
    ######################
        
    ####### bloomberght bloomberght bloomberght bloomberght bloomberght bloomberght bloomberght bloomberght bloomberght bloomberght bloomberght bloomberght #######
    
    
    def urls_bloomberght(self):    ############################33 cok fazla link var bloomberght
        main_url = "https://www.bloomberght.com/sitemap.xml" 
        soup = self.soup_maker(main_url)
        big_urls = []
    #     dom = etree.HTML(str(soup))

    #     links_element = dom.xpath('//loc/text()')
    #     print(links_element)
        urls = soup.find_all('loc')
        for url in urls:
            big_urls.append(url.text)

        # print(big_urls)
        all_urls = []
        #
        big_urls = big_urls[6:7] ##############################  azalmak için yoksa time hata verebilir bloomberght
        #
        for big in big_urls:
            soup = self.soup_maker(big)
            a_urls = soup.find_all('loc')

            for a_url in a_urls:
                all_urls.append(a_url.text)

        return all_urls




    def scrape_bloomberght_news_links(self):

        urls = self.urls_bloomberght()
        new_rows_df = pd.DataFrame(columns = ["headline", "alltext", "date", "source", "url"])


        for url in urls:
            soup = self.soup_maker(url)

            dom = etree.HTML(str(soup))

            before_new_rows_df = pd.DataFrame(columns = ["headline", "alltext", "date", "source", "url"])



            title, text, date = self.go_inside_bloomberght_links(link = url)



            new_row = {"headline": title, "alltext": text, "date": date, "source": "bloomberght", "url": url}
            before_new_rows_df = before_new_rows_df.append(new_row, ignore_index=True)

            new_rows_df = new_rows_df.append(before_new_rows_df, ignore_index=True)

        return new_rows_df




    def go_inside_bloomberght_links(self, link):
        # for link in links:
        soup = self.soup_maker(url = link)

        dom = etree.HTML(str(soup))

        title_element = dom.xpath('//div/div/section/figure/figcaption/h1/span/text()')
        if title_element != []:
            title = title_element[0]
        else:
            title = title_element
        # title = 'titlegelecekqqq'

        text_element =dom.xpath('//div/div/article/div[contains(@class, "pagebreak")]/p/text()')
        text = ''
        text_element = text_element
        for paragraph in text_element:
            text = text + paragraph

        # paragraphs = text_element.find_all('p') if text_element else []
        # text = '\n'.join(p.get_text() for p in paragraphs)
        # text= 'textgelecekq1234'

        date_element = dom.xpath('//div/div/section/figure/figcaption/div/span/time/text()')
        # print(date_element)
        date_element = list(date_element)
        try:

            date = date_element[0]
            date = date[:-7]
            date = date.rsplit(' ', 1)[0]
            # date = date.encode('utf-8')
            # date = str(date)[2:len(str(date))-1]
            date = self.turkce_aylar(date)

            date = pd.to_datetime(date)
            date = date.strftime('%Y-%d-%m-%H-%M-%S')

        except:
            date = date_element

        print(date)
        # date = '13'


        return title, text, date

    
    ######################
    

    
        
    
    
        
        
    def scrape_all_sites(self):
        # self.scrape_bloomberght_news_links()
        df_data = pd.DataFrame(columns = ["headline", "alltext", "date", "source", "url"])
        df_data = df_data.append(self.aa_news_links(), ignore_index=True)
        df_data = df_data.append(self.scrape_bloomberght_news_links(), ignore_index=True)  # 12345 azalt
        return df_data
        
        
if __name__ == '__main__':

    scraper = DataScraper()
    data = scraper.scrape_all_sites()
    old_data = pd.read_excel("yeni_türkçe_haberler.xlsx", engine = 'openpyxl')
    
    combined_data = pd.concat([old_data, data], ignore_index = True)
    
    for col in combined_data.columns:
        if any(isinstance(x, list) for x in combined_data[col]):
            combined_data[col] = combined_data[col].apply(lambda x: tuple(x) if isinstance(x, list) else x)
    final_data = combined_data.drop_duplicates()
    
    
    final_data.to_excel("yeni_türkçe_haberler.xlsx", index = False, engine = 'openpyxl')

    print(data)

https://www.aa.com.tr/tr/ekonomi/lng-talebindeki-dusus-yunanistanin-terminal-projelerini-olumsuz-etkileyebilir/3297386
2024-07-08-00-00-00
https://www.aa.com.tr/tr/ekonomi/ekonomide-gozler-enflasyonla-mucadele-adimlari-ve-buyume-verilerinde/3297379
2024-07-08-00-00-00
https://www.aa.com.tr/tr/ekonomi/kuresel-piyasalardaki-toparlanma-egilimi-surerken-vix-endeksi-27-71-ye-geriledi/3297343
2024-07-08-00-00-00
https://www.aa.com.tr/tr/ekonomi/turkiyeye-gelecek-2-5-milyar-dolarlik-dis-finansman-son-asamada/3297317
2024-07-08-00-00-00
https://www.aa.com.tr/tr/ekonomi/borsa-gune-yukselisle-basladi/3297313
2024-07-08-00-00-00
2024-01-07-00-00-00
2024-01-07-00-00-00
2024-01-07-00-00-00
2024-01-07-00-00-00
2024-01-07-00-00-00
2024-01-07-00-00-00
2024-01-07-00-00-00
2024-01-07-00-00-00
2024-01-07-00-00-00
2024-01-07-00-00-00
2024-01-07-00-00-00
2024-01-07-00-00-00
2024-01-07-00-00-00
2024-01-07-00-00-00
2024-01-07-00-00-00
2024-01-07-00-00-00
2024-01-07-00-00-00
2024-01-07-00-00-00
2024-01-07-00-

ChunkedEncodingError: ("Connection broken: InvalidChunkLength(got length b'', 0 bytes read)", InvalidChunkLength(got length b'', 0 bytes read))

In [ ]:
4+4

In [47]:
ss = '06 Ağustos 2024 07:23'

In [48]:
s = ss[:15]

In [49]:
import re

s1 = re.sub('[0-9 ]', '', s)


ing_dict = {'Ocak': 'January', 'Şubat': 'February', 'Mart': 'March', 'Nisan': 'April', 'Mayıs': 'May', 'Haziran': 'June', 'Temmuz': 'July', 'Ağustos': 'August',
    'Eylül': 'September', 'Ekim': 'October', 'Kasım': 'November', 'Aralık': 'December'}

ing_dict[s1]

'August'

In [10]:
monthDict={"01":'Jan', "02":'Feb', "03":'Mar', "04":'Apr', "05":'May', "06":'Jun', "07":'Jul', "08":'Aug',"09":'Sep', "10":'Oct', "11":'Nov', "12":'Dec'}

date = '7.08.2024'
ay_num = date.split('.')[1]

date = sda.replace(ay_num, monthDict[ay_num])

In [11]:
date

'7.Aug.2024'